# Fusión de entrenadores y LAFIFA
En este archivo se realiza la fusión de los Datasets de LAFIFA, y la acumulación de directores técnicos de Europa y Latinoamérica, el resultado será un Dataframe con la acumulación de características relacionadas con el ataque medio y defensa de los equipos deportivos más el entrenador actual según el año.




# Setting Preliminares
Importación de librerías 

In [ ]:
# carga de datos
import pandas as pd
import io
from google.colab import files

# manipulación y visualización
import matplotlib.pyplot as plt
import numpy as np
import itertools
import seaborn as sns

import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure
from numpy import corrcoef, transpose, arange
from pylab import pcolor, show, colorbar, xticks, yticks

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)
pd.options.mode.chained_assignment = None


# fechas
import datetime
import time



# Carga de Datos
Función de Google Colab para cargar datos.

In [ ]:
def upload_files ():
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    df = pd.read_csv(io.StringIO(uploaded[fn].decode('utf-8')))
    return df

In [ ]:
#cargar "fifa_caracteristicas_equipos.csv"
fifa = upload_files() 
fifa.head()

Saving fifa_caracteristicas_equipos.csv to fifa_caracteristicas_equipos.csv
User uploaded file "fifa_caracteristicas_equipos.csv" with length 478640 bytes


,equipo,liga,ataque,medio,defensa,promedio,año
0,Milan,Serie A italiana,91,91,91,91,2005
1,Arsenal,Liga Inglesa,92,92,92,92,2005
2,Real Madrid C.F.,LFP 1ª División,91,91,91,91,2005
3,Manchester United,Liga Inglesa,91,91,91,91,2005
4,Juventus,Serie A italiana,92,92,92,92,2005


In [ ]:
#Dimensiones del Dataframe fifa
fifa.shape

(10350, 7)

In [ ]:
#cargar "europa_managers"
entrenadores_europa = upload_files() 
entrenadores_europa.head()

Saving europa_managers to europa_managers
User uploaded file "europa_managers" with length 1640986 bytes


,club,fecha_activo_inicio,fecha_activo_fin,entrenador,pais,fecha_nacimiento,liga
0,Bor. Mönchengladbach,04-06-2022,30-06-2025,Daniel Farke,Alemania,30-10-1976,bundesliga
1,Bor. Mönchengladbach,01-07-2021,30-06-2022,Adi Hütter,Austria,11-02-1970,bundesliga
2,Bor. Mönchengladbach,01-07-2019,30-06-2021,Marco Rose,Alemania,11-09-1976,bundesliga
3,Bor. Mönchengladbach,22-12-2016,30-06-2019,Dieter Hecking,Alemania,12-09-1964,bundesliga
4,Bor. Mönchengladbach,21-09-2015,21-12-2016,André Schubert,Alemania,24-07-1971,bundesliga


In [ ]:
#cargar "latinoamerica_managers"
entrenadores_latinoamerica = upload_files() 
entrenadores_latinoamerica.head()

Saving latinoamerica_managers to latinoamerica_managers
User uploaded file "latinoamerica_managers" with length 337716 bytes


,club,fecha_activo_inicio,fecha_activo_fin,entrenador,pais,fecha_nacimiento,liga
0,Unión Magdalena,12-04-2022,31-12-2022,Claudio Rodríguez,Argentina,17-01-1960,betplay_dimayor
1,Unión Magdalena,17-09-2019,11-04-2022,Carlos Silva,Colombia,29-04-1973,betplay_dimayor
2,Unión Magdalena,01-07-2019,16-09-2019,Pedro Sarmiento,Colombia,26-10-1956,betplay_dimayor
3,Unión Magdalena,15-08-2017,30-06-2019,Harold Rivera,Colombia,06-07-1970,betplay_dimayor
4,Unión Magdalena,01-08-2015,31-12-2015,Carlos Silva,Colombia,29-04-1973,betplay_dimayor


# Unir datasets de managers

In [ ]:
df_entrenadores = pd.concat([entrenadores_europa ,entrenadores_latinoamerica], axis=0)
df_entrenadores.reset_index(drop = True, inplace = True) # reiniciar index
df_entrenadores

,club,fecha_activo_inicio,fecha_activo_fin,entrenador,pais,fecha_nacimiento,liga
0,Bor. Mönchengladbach,04-06-2022,30-06-2025,Daniel Farke,Alemania,30-10-1976,bundesliga
1,Bor. Mönchengladbach,01-07-2021,30-06-2022,Adi Hütter,Austria,11-02-1970,bundesliga
2,Bor. Mönchengladbach,01-07-2019,30-06-2021,Marco Rose,Alemania,11-09-1976,bundesliga
3,Bor. Mönchengladbach,22-12-2016,30-06-2019,Dieter Hecking,Alemania,12-09-1964,bundesliga
4,Bor. Mönchengladbach,21-09-2015,21-12-2016,André Schubert,Alemania,24-07-1971,bundesliga
...,...,...,...,...,...,...,...
23301,Armenio,01-01-2022,31-12-2022,Mario Gómez,Argentina,27-02-1957,primera_division_argentina
23302,Armenio,01-01-2018,30-06-2019,José Villarreal,Argentina,19-06-1964,primera_division_argentina
23303,Armenio,01-07-2014,31-12-2015,Federico Domínguez,Argentina,13-08-1976,primera_division_argentina
23304,Armenio,01-03-2010,30-06-2013,Fernando Ruíz,Argentina,26-04-1972,primera_division_argentina


Arreglar años de dataset fifa

In [ ]:
años=[]
for año in list(range(len(fifa))):
  fecha=fifa['año'][año]
  años.append('30-06-'+str(fecha)) 

fifa['año']=años

In [ ]:
fifa

,equipo,liga,ataque,medio,defensa,promedio,año
0,Milan,Serie A italiana,91,91,91,91,30-06-2005
1,Arsenal,Liga Inglesa,92,92,92,92,30-06-2005
2,Real Madrid C.F.,LFP 1ª División,91,91,91,91,30-06-2005
3,Manchester United,Liga Inglesa,91,91,91,91,30-06-2005
4,Juventus,Serie A italiana,92,92,92,92,30-06-2005
...,...,...,...,...,...,...,...
10345,Central Coast Mariners,Australia A-League (1),62,62,62,62,30-06-2022
10346,Brisbane Roar,Australia A-League (1),61,61,61,61,30-06-2022
10347,Rochdale,England League Two (4),61,61,61,61,30-06-2022
10348,Harrogate Town,England League Two (4),61,61,61,61,30-06-2022


In [ ]:
#funcion para quitar tildes y ñ
def quitar_tildes(x):
  x = x.replace('á','a')
  x = x.replace('é', 'e')
  x = x.replace('í', 'i')
  x = x.replace('ó', 'o')
  x = x.replace('ú', 'u')
  x = x.replace('Á','A')
  x = x.replace('É', 'E')
  x = x.replace('Í', 'I')
  x = x.replace('Ó', 'O')
  x = x.replace('Ú', 'U')
  x = x.replace('ã','a')
  x = x.replace('ê','e')
  x = x.replace('ñ','n')
  x = x.replace('Ñ','N')
  return x

Creación de nueva columna en el dataframe “fifa”con información de los entrenadores.

In [ ]:
entrenador_equipo = [None]*len(fifa)

#Recorrer entrenadores
for entrenador in list(range(len(df_entrenadores))):
  print(entrenador)
#for entrenador in list(range(2)):
  #Fecha entrenador inicio
  fecha_entrenador_inicio=(df_entrenadores['fecha_activo_inicio'][entrenador])
  fecha_entrenador_inicio_lista=fecha_entrenador_inicio.replace('-','/')
  fecha_entrenador_inicio_formatted_date = time.strptime(fecha_entrenador_inicio_lista, "%d/%m/%Y")
  #Fecha  entrenador fin
  fecha_entrenador_fin=(df_entrenadores['fecha_activo_fin'][entrenador])
  fecha_entrenador_fin_lista=fecha_entrenador_fin.replace('-','/')
  fecha_entrenador_fin_formatted_date = time.strptime(fecha_entrenador_fin_lista, "%d/%m/%Y")
  for club in list(range(len(fifa))):
    x = fifa['equipo'][club]
    y = df_entrenadores['club'][entrenador]
    #quitar caracteres especiales
    x = ''.join(filter(str.isalnum, x))
    y = ''.join(filter(str.isalnum, y))
    #quitar tildes y simbolos extraños
    x = quitar_tildes(x)
    y = quitar_tildes(y)
    #quitar mayusculas
    x = x.lower()
    y = y.lower()
    #Comparar club del equipo con club del entrenador
    if x in y or y in x:
      #Fecha club
      fecha_club=(fifa['año'][club])
      fecha_club_lista=fecha_club.replace('-','/')
      fecha_club_formatted_date = time.strptime(fecha_club_lista, "%d/%m/%Y")
      #Comprovar si la fecha del equipo se encuentra en el rango de fechas del entrenador
      if fecha_club_formatted_date >= fecha_entrenador_inicio_formatted_date and fecha_club_formatted_date <= fecha_entrenador_fin_formatted_date:
        #asignar a la lista del entrenador en la posicion actual el valor del entrenador
        entrenador_equipo[club] = df_entrenadores['entrenador'][entrenador]

fifa['entrenador']=entrenador_equipo

In [ ]:
# ejeplo del dataframe fifa con el equipo FC Barcelona
fifa[fifa['equipo']=='FC Barcelona']

,equipo,liga,ataque,medio,defensa,promedio,año,entrenador
4060,FC Barcelona,Liga BBVA,86,86,86,86,30-06-2013,Vilanova
7642,FC Barcelona,LaLiga Santander,87,87,87,87,30-06-2019,Valverde
8294,FC Barcelona,LaLiga Santander,88,88,88,88,30-06-2020,Quique Setién
9014,FC Barcelona,LaLiga Santander,84,84,84,84,30-06-2021,Ronald Koeman
9697,FC Barcelona,Spain Primera División (1),83,83,83,83,30-06-2022,Xavi


In [ ]:
#Eliminar filas con valores perdidos en el dataframe fifa. 
fifa_limpio=fifa.dropna(axis=0)

In [ ]:
fifa_limpio.shape

(4515, 8)

In [ ]:
#Porcentaje de datos perdidos del dataset fifa
for col in fifa_limpio.columns:
    pct_missing = np.mean(fifa_limpio[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

equipo - 0%
liga - 0%
ataque - 0%
medio - 0%
defensa - 0%
promedio - 0%
año - 0%
entrenador - 0%


# Generar Salida del Dataframe

In [ ]:
#Descargar dataframe
from google.colab import files

with open('fifa_caracteristicas_equipos_con_entrenadores.csv', 'w') as f:
  fifa_limpio.to_csv(f,  index = False)
  

files.download('fifa_caracteristicas_equipos_con_entrenadores.csv')

fifa_limpio.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,equipo,liga,ataque,medio,defensa,promedio,año,entrenador
0,Milan,Serie A italiana,91,91,91,91,30-06-2005,Carlo Ancelotti
1,Arsenal,Liga Inglesa,92,92,92,92,30-06-2005,Jorge Burruchaga
2,Real Madrid C.F.,LFP 1ª División,91,91,91,91,30-06-2005,Vanderlei Luxemburgo
3,Manchester United,Liga Inglesa,91,91,91,91,30-06-2005,Alex Ferguson
4,Juventus,Serie A italiana,92,92,92,92,30-06-2005,Fabio Capello
